In [1]:
import numpy as np
import os
import onnx
from onnxruntime_extensions import get_library_path, PyOp, onnx_op, PyOrtFunction
from qonnx.core.datatype import DataType
from qonnx.core.modelwrapper import ModelWrapper
from onnx.helper import make_attribute
from driver_base import FINNExampleOverlay
from driver import io_shape_dict
set_pyop = False

platform = "zynq-iodma"
batch_size = 1
bitfile = "finn-accel.bit"
outputfile = "output.npy"
runtime_weight_dir = "runtime_weights/"

# instantiate FINN accelerator driver and pass batchsize and bitfile
accel = FINNExampleOverlay(
    bitfile_name = bitfile, platform = platform,
    io_shape_dict = io_shape_dict, batch_size = batch_size,
    runtime_weight_dir = runtime_weight_dir
)

if not set_pyop:
    # Implement the CustomOp by decorating a function with onnx_op
    @onnx_op(op_type="StreamingDataflowPartition", inputs=[PyOp.dt_float], outputs=[PyOp.dt_float])
    def StreamingDataflowPartition(inputs):
        obuf_normal = accel.execute(inputs)
        return obuf_normal.astype(np.float32)
    set_pyop = True
print(set_pyop)

True


In [3]:
from qonnx.core.onnx_exec import execute_onnx 
model = ModelWrapper("dataflow_parent.onnx")
DOMAIN = "qonnx.custom_op.fpgadataflow"

for n in model.graph.node:
    if n.domain == "finn.custom_op.fpgadataflow":
        n.domain = DOMAIN
#     if n.op_type == "StreamingDataflowPartition":
#         n.name = ''
#         n.attribute.append(make_attribute("bitfile","finn-accel.bit"))
        
model.check_all_tensor_shapes_specified()
execute_onnx(model, {'global_in':np.load("dog.npy")})


TypeError: '_Environ' object is not callable

In [ ]:
inp = np.load("dog.npy")

model_func = PyOrtFunction.from_model("./dataflow_parent_updated.onnx")
outputs = model_func(inp)

In [12]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(10,20,[1,3,224,224]).astype(np.float32)
# x = np.([1,3,224,224]).astype(np.float32)
np.save("dog.npy",x)

In [30]:
import numpy as np
import os
import onnx
from onnxruntime_extensions import get_library_path, PyOp, onnx_op, PyOrtFunction
from qonnx.core.datatype import DataType
from qonnx.core.modelwrapper import ModelWrapper
from onnx.helper import make_attribute
from driver_base import FINNExampleOverlay
from driver import io_shape_dict

platform = "zynq-iodma"
batch_size = 1
bitfile = "finn-accel.bit"
outputfile = "output.npy"
runtime_weight_dir = "runtime_weights/"

accel = FINNExampleOverlay(
    bitfile_name = bitfile, platform = platform,
    io_shape_dict = io_shape_dict, batch_size = batch_size,
    runtime_weight_dir = runtime_weight_dir
)

def StreamingDataflowPartition(inputs):
    print("EXECUTE SDP")
    obuf_normal = accel.execute(inputs)
#     print(obuf_normal)
    return obuf_normal.astype(np.float32)
x = np.load("dog.npy")
x.shape
x = np.moveaxis(x,1,-1)
res = StreamingDataflowPartition(x)


EXECUTE SDP


In [1]:
! python op_test.py

{'idt': [UINT8], 'odt': [INT16], 'ishape_normal': [(1, 224, 224, 3)], 'oshape_normal': [(1, 14, 14, 512)], 'ishape_folded': [(1, 224, 224, 3, 1)], 'oshape_folded': [(1, 14, 14, 32, 16)], 'ishape_packed': [(1, 224, 224, 3, 1)], 'oshape_packed': [(1, 14, 14, 32, 32)], 'input_dma_name': ['idma0'], 'output_dma_name': ['odma0'], 'number_of_external_weights': 0, 'num_inputs': 1, 'num_outputs': 1}
Exception ignored in: <function PynqBuffer.__del__ at 0xffff95030700>
Traceback (most recent call last):
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/buffer.py", line 63, in __del__
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/buffer.py", line 82, in freebuffer
TypeError: 'NoneType' object is not callable
Exception ignored in: <function PynqBuffer.__del__ at 0xffff95030700>
Traceback (most recent call last):
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/buffer.py", line 63, in __del__
  File "/usr/local/share/pynq-venv/lib/python